In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
drive_path = 'C:/Users/shiu/Desktop/206computer/10366021/code/'
#modelset = "1000"

In [4]:
# encoding: utf-8
import sys
sys.path.append('C:/Users/shiu/Desktop/206computer/10366021/code')
sys.path.append('C:/Users/shiu/Desktop/206computer/10366021/code/keras_position_wise_feed_forward')
sys.path.append('C:/Users/shiu/Desktop/206computer/10366021/code/tensorflow_fast_attention')
sys.path.append('C:/Users/shiu/Desktop/206computer/10366021/code/keras_performer')
import unittest
import numpy as np
import math
from keras_performer import performer_mask as pfr

In [5]:
def loadDictionary(file):
    import pickle
    a_file = open(file, "rb")
    dt = pickle.load(a_file)
    return dt

def load(model_name):
    from keras_performer import performer_mask
    from tensorflow import keras
    from keras_embed_sim import EmbeddingRet, EmbeddingSim
    from keras_pos_embd import TrigPosEmbedding
    from tensorflow_fast_attention.fast_attention_2 import  softmax_kernel_transformation,Attention, SelfAttention    
    from keras_position_wise_feed_forward.feed_forward import FeedForward  

    co = performer_mask.get_custom_objects()  #需要更正是哪一版的performer
    co["softmax_kernel_transformation"] = softmax_kernel_transformation #新增softmax_kernel_transformation
    
    model = keras.models.load_model(model_name, custom_objects= co)
    s = loadDictionary(drive_path + 'train_data_reduced/source_token_dict.pickle')
    t = loadDictionary(drive_path + 'train_data_reduced/target_token_dict.pickle')
    t_inv = loadDictionary(drive_path + 'train_data_reduced/target_token_dict_inv.pickle')
    return model, s, t, t_inv
    
def decomposeModel(model):
    encoder_model = model.get_layer('encoder_model')
    classifier_decoder_model = model.get_layer('classifier_decoder_model')
    print(encoder_model.summary())
    print(classifier_decoder_model.summary())
    return encoder_model, classifier_decoder_model
        
def loadMaxLen(filename):     
    with open(filename) as f:
        lines = f.readlines()
        return lines
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

# Train

In [6]:
class TestTranslate(unittest.TestCase):

    def test_translate(self):
        
        #model, source_token_dict, target_token_dict, target_token_dict_inv = load(drive_path + "test_model_" + modelset + ".h5")
        
        model, source_token_dict, target_token_dict, target_token_dict_inv = load('C:/Users/shiu/Desktop/result/model/again3_checkpoint_1000.h5') 
        # Predict
        #load traing/test data
        source_max_len_loaded = loadMaxLen(drive_path+'train_data_reduced/source_max_len.txt')
        max_seq_len = int(source_max_len_loaded[0])       
        #print('source_max_len_loaded : ', source_max_len_loaded)
        #print('max_seq_len : ',max_seq_len)
        encode_input = []
        decode_input = []
        decode_output1 = []
        decode_output2 = []
                    
        encode_input.append(loadTestTrainData(drive_path + "train_data_reduced/x_train[0]_0.npy"))
        print('encode_input : ',len(encode_input))
        decode_input.append(loadTestTrainData(drive_path + "train_data_reduced/x_train[1]_0.npy"))        
        decode_output1.append(loadTestTrainData(drive_path + "train_data_reduced/y_train[0]_0.npy"))
        decode_output2.append(loadTestTrainData(drive_path + "train_data_reduced/y_train[1]_0.npy"))
        '''
        print("encode_input_loaded[0].shape:", encode_input_loaded[0].shape)
        print("decode_input_loaded[0].shape:", decode_input_loaded[0].shape)
        print("decode_output1[0].shape:", decode_output1[0].shape)
        print("decode_output2_loaded[0].shape:", decode_output2_loaded[0].shape)

        print("encode_input_loaded len:", len(encode_input_loaded))
        print("decode_input_loaded len:", len(decode_input_loaded))
        print("decode_output1 len:", len(decode_output1))
        print("decode_output2_loaded len:", len(decode_output2_loaded))
        
        print("encode_input_loaded[0] len:", len(encode_input_loaded[0]))
        print("decode_input_loaded[0] len:", len(decode_input_loaded[0]))
        print("decode_output1[0] len:", len(decode_output1[0]))
        print("decode_output2_loaded[0] len:", len(decode_output2_loaded[0]))
        
        print("encode_input_loaded[0][0] len:", len(encode_input_loaded[0][0]))
        print("decode_input_loaded[0][0] len:", len(decode_input_loaded[0][0]))
        print("decode_output1[0][0] len:", len(decode_output1[0][0]))
        print("decode_output2_loaded[0][0] len:", len(decode_output2_loaded[0][0]))    
        
        encode_input = []
        decode_input = []
        decode_output2 = []
        
        d_two = [] 
        for i in range(len(encode_input_loaded[0])):          
            d_two.append(encode_input_loaded[0][i][0:max_seq_len])              
        #switch to np array
        d_two = np.asarray(d_two)
        #give np array
        encode_input.append(d_two)
        #switch to np array
        #encode_input = np.array(encode_input)
        
        d_two = []
        for i in range(len(decode_input_loaded[0])):          
            d_two.append(decode_input_loaded[0][i][0:max_seq_len])  
        #switch to np array
        d_two = np.asarray(d_two)
        #give np array
        decode_input.append(d_two)
        #switch to np array
        #decode_input = np.array(decode_input)
        
        d_two = []
        for i in range(len(decode_output2_loaded[0])):          
            d_two.append(decode_output2_loaded[0][i][0:max_seq_len])  
        #switch to np array
        d_two = np.asarray(d_two)
        #give np array
        decode_output2.append(d_two)
        #switch to np array
        #decode_output2 = np.array(decode_output2, dtype=object)
        
        print("encode_input[0].shape:", encode_input[0].shape)
        print("decode_input[0].shape:", decode_input[0].shape)
        print("decode_output1[0].shape:", decode_output1[0].shape)
        print("decode_output2[0].shape:", decode_output2[0].shape)
        '''
        #recover the target tokens for training data
        decode_output2[0] = np.squeeze(decode_output2[0])
        decode_output2_list = decode_output2[0].tolist()
        target_train_tokens = [ [ target_token_dict_inv[token] for token in sample if target_token_dict_inv[token] not in ['<END>', '<PAD>']] for sample in decode_output2_list ] 
      
        source_max_len = max_seq_len
        
        encode_input = encode_input[0] 
        print('encode_input : ',encode_input)
        result_err = []
        result_dec = []   
        data_size = 1001
        
        for i in range(math.ceil(encode_input.shape[0]/data_size)):
            lmmodelans, errortypes, decoded = pfr.decode(
                model,# 拆解成encoder ,decoder
                encode_input.tolist()[i*data_size:(i+1)*data_size], #[code1, code2]==> ['class', 'A',...], ['class' 'B' ...]
                start_token=target_token_dict['<START>'],
                end_token=target_token_dict['<END>'],
                pad_token=target_token_dict['<PAD>'],
                max_len=source_max_len
            )
            #[dec1, dec2] like[ ['class', 'A',...], ['class' 'B' ...] ]
            result_err.append(errortypes)
            result_dec.append(decoded)
        '''   
        print("len(result_err):", len(result_err))
        print("len(result_dec):", len(result_dec))
        print("len(result_err[0]):", len(result_err[0]))
        print("len(result_dec[0]):", len(result_dec[0]))    
        '''
        Gate = 0.5

        #TypeOutput = drive_path + modelset + '_train_type.txt' #錯誤類型輸出
        TypeOutput = drive_path+'meteor/' + 'checkpoint_'+ 1000 + '_train_type.txt'

        with open(TypeOutput, 'w') as f :
            Sum_Recall = 0.0
            Sum_Precision = 0.0
            Sum_F_Score = 0.0
                     
            for i in range(len(result_err)):# error classify
                errortypes_gate = np.array([list(map(lambda e: 1 if np.sign(e-Gate)>0 else 0, t)) for t in result_err[i]])
                
                if encode_input.shape[0] % data_size == 0: 
                    result_err_len = data_size
                else:
                    result_err_len = data_size if i < encode_input.shape[0] // data_size else encode_input.shape[0] % data_size
                    
                for j in range(result_err_len):                                
                    Num = "===== " + str(i*data_size+j) + " ====="
                    print(Num)
                    f.write(Num+"\n")
   
                    std = "=====標準答案=====" + str(decode_output1[0][i*data_size+j])
                    print(std)
                    f.write(std+"\n")

                    pred = "=====預測答案=====" + str(errortypes_gate[j])
                    print(pred)
                    f.write(pred+"\n")

                    precision = metrics.precision_score(decode_output1[0][i*data_size+j], errortypes_gate[j])
                    Sum_Precision += precision
                    recall = metrics.recall_score(decode_output1[0][i*data_size+j], errortypes_gate[j])
                    Sum_Recall += recall
                    f_score = metrics.f1_score(decode_output1[0][i*data_size+j], errortypes_gate[j])
                    Sum_F_Score += f_score

                    precision_print = "Precision : " + str(precision)
                    print(precision_print)
                    f.write(precision_print+"\n")

                    recall_print = "Recall : " + str(recall)
                    print(recall_print)
                    f.write(recall_print+"\n")

                    f_score_print = "F score : " + str(f_score)
                    print(f_score_print)
                    f.write(f_score_print+"\n")

                    print(" ")
                    f.write("\n")

            Avg_Precision = round(Sum_Precision/encode_input.shape[0], 4) #平均預測
            Avg_Recall = round(Sum_Recall/encode_input.shape[0], 4) #平均召回
            Avg_F_Score = round(Sum_F_Score/encode_input.shape[0], 4) #平均 F 值

            Avg_Precision_print = "Avg_Precision : " + str(Avg_Precision)
            print(Avg_Precision_print)
            f.write(Avg_Precision_print+"\n")

            Avg_Recall_print = "Avg_Recall : " + str(Avg_Recall)
            print(Avg_Recall_print)
            f.write(Avg_Recall_print+"\n")

            Avg_F_Score_print = "Avg_F_Score : " + str(Avg_F_Score)
            print(Avg_F_Score_print)
            f.write(Avg_F_Score_print+"\n")

            print(" ")
            f.write("\n")

        #MessOutput = drive_path + modelset + '_train_meteor.txt' #錯誤訊息輸出
        MessOutput = drive_path  + 'checkpoint_'+ modelset + '_train_meteor.txt'

        with open(MessOutput, 'w') as f :
            Sum_Meteor = 0.0
            for i in range(len(result_dec)): #for error messages OK!
                if encode_input.shape[0] % data_size == 0: 
                    result_dec_len = data_size
                else:
                    result_dec_len = data_size if i < encode_input.shape[0] // data_size else encode_input.shape[0] % data_size
                for j in range(result_dec_len):
                    Num = "===== " + str(i*data_size+j) + " ====="
                    print(Num)
                    f.write(Num+"\n")

                    standard = ''.join(target_train_tokens[i*data_size+j])
                    standard_print = "=====標準答案=====" + standard
                    ps_standard = parseSentence(standard)
                    print(standard_print)
                    f.write(standard_print+"\n")

                    predicted = ''.join(map(lambda x: target_token_dict_inv[x], result_dec[i][j][1:-1]))
                    predicted_print = "=====預測答案=====" + predicted
                    ps_predicted = parseSentence(predicted)
                    print(predicted_print)
                    f.write(predicted_print+"\n")

                    Meteor_Score = round(meteor.meteor_score([str(ps_standard)], str(ps_predicted)), 4)
                    Sum_Meteor += Meteor_Score
                    Meteor_Score_print = "Meteor_Score : " + str(Meteor_Score)
                    print(Meteor_Score_print)
                    f.write(Meteor_Score_print+"\n")

                    print(" ")
                    f.write("\n")
                
            Avg_Meteor = round(Sum_Meteor/encode_input.shape[0],4)
            Avg_Meteor_print = "Avg_Meteor_Score : " + str(Avg_Meteor)
            print(Avg_Meteor_print)
            f.write(Avg_Meteor_print+"\n")
        
x=TestTranslate()
x.test_translate()

--------------------------------------------------------------------
output: Tensor("cast_mask/SelectV2:0", shape=(None, 1001), dtype=float32)
--------------------------------------------------------------------
--------------------------------------------------------------------
attention_output shape: (None, 1001, 32)
attention again
--------------------------------------------------------------------
--------------------------------------------------------------------
attention_output shape: (None, 1001, 32)
attention again
--------------------------------------------------------------------
--------------------------------------------------------------------
attention_output shape: (None, 1001, 32)
attention again
--------------------------------------------------------------------
--------------------------------------------------------------------
attention_output shape: (None, 1001, 32)
attention again
--------------------------------------------------------------------
--------

ResourceExhaustedError:  OOM when allocating tensor with shape[2,776,1428] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/Decoder-Output/Softmax
 (defined at C:\Users\shiu\anaconda3\envs\python3711\lib\site-packages\keras\activations.py:80)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_predict_function_341320]

Errors may have originated from an input operation.
Input Source operations connected to node model/Decoder-Output/Softmax:
In[0] model/Decoder-Output/BiasAdd (defined at C:\Users\shiu\anaconda3\envs\python3711\lib\site-packages\keras\backend.py:5976)

Operation defined at: (most recent call last)
>>>   File "C:\Users\shiu\anaconda3\envs\python3711\lib\runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>

# Validation

In [ ]:
class TestTranslate(unittest.TestCase):

    def test_translate(self):

        #model, source_token_dict, target_token_dict, target_token_dict_inv = load(drive_path + "test_model_" + modelset + ".h5")
        model, source_token_dict, target_token_dict, target_token_dict_inv = load(drive_path + "checkpoint_" + modelset + ".h5")
        # Predict
        #load traing/test data 
        source_max_len_loaded = loadMaxLen(drive_path + "source_max_len.txt")
        max_seq_len = int(source_max_len_loaded[0])       
        
        encode_input = []
        decode_input = []
        decode_output1 = []
        decode_output2 = []
                    
        encode_input.append(loadTestTrainData(drive_path + "x_validation[0]_0.npy"))
        decode_input.append(loadTestTrainData(drive_path + "x_validation[1]_0.npy"))        
        decode_output1.append(loadTestTrainData(drive_path + "y_validation[0]_0.npy"))
        decode_output2.append(loadTestTrainData(drive_path + "y_validation[1]_0.npy"))
        '''
        print("encode_input_loaded[0].shape:", encode_input_loaded[0].shape)
        print("decode_input_loaded[0].shape:", decode_input_loaded[0].shape)
        print("decode_output1[0].shape:", decode_output1[0].shape)
        print("decode_output2_loaded[0].shape:", decode_output2_loaded[0].shape)

        print("encode_input_loaded len:", len(encode_input_loaded))
        print("decode_input_loaded len:", len(decode_input_loaded))
        print("decode_output1 len:", len(decode_output1))
        print("decode_output2_loaded len:", len(decode_output2_loaded))
        
        print("encode_input_loaded[0] len:", len(encode_input_loaded[0]))
        print("decode_input_loaded[0] len:", len(decode_input_loaded[0]))
        print("decode_output1[0] len:", len(decode_output1[0]))
        print("decode_output2_loaded[0] len:", len(decode_output2_loaded[0]))
        
        print("encode_input_loaded[0][0] len:", len(encode_input_loaded[0][0]))
        print("decode_input_loaded[0][0] len:", len(decode_input_loaded[0][0]))
        print("decode_output1[0][0] len:", len(decode_output1[0][0]))
        print("decode_output2_loaded[0][0] len:", len(decode_output2_loaded[0][0]))    
        
        encode_input = []
        decode_input = []
        decode_output2 = []
        
        d_two = [] 
        for i in range(len(encode_input_loaded[0])):          
            d_two.append(encode_input_loaded[0][i][0:max_seq_len])              
        #switch to np array
        d_two = np.asarray(d_two)
        #give np array
        encode_input.append(d_two)
        #switch to np array
        #encode_input = np.array(encode_input)
        
        d_two = []
        for i in range(len(decode_input_loaded[0])):          
            d_two.append(decode_input_loaded[0][i][0:max_seq_len])  
        #switch to np array
        d_two = np.asarray(d_two)
        #give np array
        decode_input.append(d_two)
        #switch to np array
        #decode_input = np.array(decode_input)
        
        d_two = []
        for i in range(len(decode_output2_loaded[0])):          
            d_two.append(decode_output2_loaded[0][i][0:max_seq_len])  
        #switch to np array
        d_two = np.asarray(d_two)
        #give np array
        decode_output2.append(d_two)
        #switch to np array
        #decode_output2 = np.array(decode_output2, dtype=object)
        
        print("encode_input[0].shape:", encode_input[0].shape)
        print("decode_input[0].shape:", decode_input[0].shape)
        print("decode_output1[0].shape:", decode_output1[0].shape)
        print("decode_output2[0].shape:", decode_output2[0].shape)
        '''
        #recover the target tokens for training data
        decode_output2[0] = np.squeeze(decode_output2[0])
        decode_output2_list = decode_output2[0].tolist()
        target_validation_tokens = [ [ target_token_dict_inv[token] for token in sample if target_token_dict_inv[token] not in ['<END>', '<PAD>']] for sample in decode_output2_list ] 
      
        source_max_len = max_seq_len
        
        encode_input = encode_input[0] 
        
        result_err = []
        result_dec = []   
        data_size = 512
        
        for i in range(math.ceil(encode_input.shape[0]/data_size)):
            errortypes, decoded = pfr.decode(
                model,
                encode_input.tolist()[i*data_size:(i+1)*data_size], #[code1, code2]==> ['class', 'A',...], ['class' 'B' ...]
                start_token=target_token_dict['<START>'],
                end_token=target_token_dict['<END>'],
                pad_token=target_token_dict['<PAD>'],
                max_len=source_max_len
            )
            #[dec1, dec2] like[ ['class', 'A',...], ['class' 'B' ...] ]
            result_err.append(errortypes)
            result_dec.append(decoded)
        '''   
        print("len(result_err):", len(result_err))
        print("len(result_dec):", len(result_dec))
        print("len(result_err[0]):", len(result_err[0]))
        print("len(result_dec[0]):", len(result_dec[0]))    
        '''
        Gate = 0.5

        #TypeOutput = drive_path + modelset + '_validation_type.txt' #錯誤類型輸出
        TypeOutput = drive_path + 'checkpoint_'+ modelset + '_validation_type.txt'

        with open(TypeOutput, 'w') as f :
            Sum_Recall = 0.0
            Sum_Precision = 0.0
            Sum_F_Score = 0.0
                     
            for i in range(len(result_err)):# error classify
                errortypes_gate = np.array([list(map(lambda e: 1 if np.sign(e-Gate)>0 else 0, t)) for t in result_err[i]])
                
                if encode_input.shape[0] % data_size == 0: 
                    result_err_len = data_size
                else:
                    result_err_len = data_size if i < encode_input.shape[0] // data_size else encode_input.shape[0] % data_size
                    
                for j in range(result_err_len):                                
                    Num = "===== " + str(i*data_size+j) + " ====="
                    print(Num)
                    f.write(Num+"\n")
   
                    std = "=====標準答案=====" + str(decode_output1[0][i*data_size+j])
                    print(std)
                    f.write(std+"\n")

                    pred = "=====預測答案=====" + str(errortypes_gate[j])
                    print(pred)
                    f.write(pred+"\n")

                    precision = metrics.precision_score(decode_output1[0][i*data_size+j], errortypes_gate[j])
                    Sum_Precision += precision
                    recall = metrics.recall_score(decode_output1[0][i*data_size+j], errortypes_gate[j])
                    Sum_Recall += recall
                    f_score = metrics.f1_score(decode_output1[0][i*data_size+j], errortypes_gate[j])
                    Sum_F_Score += f_score

                    precision_print = "Precision : " + str(precision)
                    print(precision_print)
                    f.write(precision_print+"\n")

                    recall_print = "Recall : " + str(recall)
                    print(recall_print)
                    f.write(recall_print+"\n")

                    f_score_print = "F score : " + str(f_score)
                    print(f_score_print)
                    f.write(f_score_print+"\n")

                    print(" ")
                    f.write("\n")

            Avg_Precision = round(Sum_Precision/encode_input.shape[0], 4) #平均預測
            Avg_Recall = round(Sum_Recall/encode_input.shape[0], 4) #平均召回
            Avg_F_Score = round(Sum_F_Score/encode_input.shape[0], 4) #平均 F 值

            Avg_Precision_print = "Avg_Precision : " + str(Avg_Precision)
            print(Avg_Precision_print)
            f.write(Avg_Precision_print+"\n")

            Avg_Recall_print = "Avg_Recall : " + str(Avg_Recall)
            print(Avg_Recall_print)
            f.write(Avg_Recall_print+"\n")

            Avg_F_Score_print = "Avg_F_Score : " + str(Avg_F_Score)
            print(Avg_F_Score_print)
            f.write(Avg_F_Score_print+"\n")

            print(" ")
            f.write("\n")

        #MessOutput = drive_path + modelset + '_validation_meteor.txt' #錯誤訊息輸出
        MessOutput = drive_path + 'checkpoint_'+ modelset + '_validation_meteor.txt'

        with open(MessOutput, 'w') as f :
            Sum_Meteor = 0.0
            for i in range(len(result_dec)): #for error messages OK!
                if encode_input.shape[0] % data_size == 0: 
                    result_dec_len = data_size
                else:
                    result_dec_len = data_size if i < encode_input.shape[0] // data_size else encode_input.shape[0] % data_size
                for j in range(result_dec_len):
                    Num = "===== " + str(i*data_size+j) + " ====="
                    print(Num)
                    f.write(Num+"\n")

                    standard = ''.join(target_validation_tokens[i*data_size+j])
                    standard_print = "=====標準答案=====" + standard
                    ps_standard = parseSentence(standard)
                    print(standard_print)
                    f.write(standard_print+"\n")

                    predicted = ''.join(map(lambda x: target_token_dict_inv[x], result_dec[i][j][1:-1]))
                    predicted_print = "=====預測答案=====" + predicted
                    ps_predicted = parseSentence(predicted)
                    print(predicted_print)
                    f.write(predicted_print+"\n")

                    Meteor_Score = round(meteor.meteor_score([str(ps_standard)], str(ps_predicted)), 4)
                    Sum_Meteor += Meteor_Score
                    Meteor_Score_print = "Meteor_Score : " + str(Meteor_Score)
                    print(Meteor_Score_print)
                    f.write(Meteor_Score_print+"\n")

                    print(" ")
                    f.write("\n")
                
            Avg_Meteor = round(Sum_Meteor/encode_input.shape[0],4)
            Avg_Meteor_print = "Avg_Meteor_Score : " + str(Avg_Meteor)
            print(Avg_Meteor_print)
            f.write(Avg_Meteor_print+"\n")
        
x=TestTranslate()
x.test_translate()

# Test

In [ ]:
class TestTranslate(unittest.TestCase):

    def test_translate(self):

        #model, source_token_dict, target_token_dict, target_token_dict_inv = load(drive_path + "test_model_" + modelset + ".h5")
        model, source_token_dict, target_token_dict, target_token_dict_inv = load(drive_path + "checkpoint_" + modelset + ".h5")
        # Predict
        #load traing/test data 
        source_max_len_loaded = loadMaxLen(drive_path + "source_max_len.txt")
        max_seq_len = int(source_max_len_loaded[0])       
        
        encode_input = []
        decode_input = []
        decode_output1 = []
        decode_output2 = []
                    
        encode_input.append(loadTestTrainData(drive_path + "x_test[0]_0.npy"))
        decode_input.append(loadTestTrainData(drive_path + "x_test[1]_0.npy"))        
        decode_output1.append(loadTestTrainData(drive_path + "y_test[0]_0.npy"))
        decode_output2.append(loadTestTrainData(drive_path + "y_test[1]_0.npy"))  
        
        '''
        print("encode_input_loaded[0].shape:", encode_input_loaded[0].shape)
        print("decode_input_loaded[0].shape:", decode_input_loaded[0].shape)
        print("decode_output1[0].shape:", decode_output1[0].shape)
        print("decode_output2_loaded[0].shape:", decode_output2_loaded[0].shape)

        print("encode_input_loaded len:", len(encode_input_loaded))
        print("decode_input_loaded len:", len(decode_input_loaded))
        print("decode_output1 len:", len(decode_output1))
        print("decode_output2_loaded len:", len(decode_output2_loaded))
        
        print("encode_input_loaded[0] len:", len(encode_input_loaded[0]))
        print("decode_input_loaded[0] len:", len(decode_input_loaded[0]))
        print("decode_output1[0] len:", len(decode_output1[0]))
        print("decode_output2_loaded[0] len:", len(decode_output2_loaded[0]))
        
        print("encode_input_loaded[0][0] len:", len(encode_input_loaded[0][0]))
        print("decode_input_loaded[0][0] len:", len(decode_input_loaded[0][0]))
        print("decode_output1[0][0] len:", len(decode_output1[0][0]))
        print("decode_output2_loaded[0][0] len:", len(decode_output2_loaded[0][0]))    
        
        encode_input = []
        decode_input = []
        decode_output2 = []
        
        d_two = [] 
        for i in range(len(encode_input_loaded[0])):          
            d_two.append(encode_input_loaded[0][i][0:max_seq_len])              
        #switch to np array
        d_two = np.asarray(d_two)
        #give np array
        encode_input.append(d_two)
        #switch to np array
        #encode_input = np.array(encode_input)
        
        d_two = []
        for i in range(len(decode_input_loaded[0])):          
            d_two.append(decode_input_loaded[0][i][0:max_seq_len])  
        #switch to np array
        d_two = np.asarray(d_two)
        #give np array
        decode_input.append(d_two)
        #switch to np array
        #decode_input = np.array(decode_input)
        
        d_two = []
        for i in range(len(decode_output2_loaded[0])):          
            d_two.append(decode_output2_loaded[0][i][0:max_seq_len])  
        #switch to np array
        d_two = np.asarray(d_two)
        #give np array
        decode_output2.append(d_two)
        #switch to np array
        #decode_output2 = np.array(decode_output2, dtype=object)
        
        print("encode_input[0].shape:", encode_input[0].shape)
        print("decode_input[0].shape:", decode_input[0].shape)
        print("decode_output1[0].shape:", decode_output1[0].shape)
        print("decode_output2[0].shape:", decode_output2[0].shape)
        '''
        #recover the target tokens for training data
        decode_output2[0] = np.squeeze(decode_output2[0])
        decode_output2_list = decode_output2[0].tolist()
        target_test_tokens = [ [ target_token_dict_inv[token] for token in sample if target_token_dict_inv[token] not in ['<END>', '<PAD>']] for sample in decode_output2_list ] 
      
        source_max_len = max_seq_len
        
        encode_input = encode_input[0] 
        
        result_err = []
        result_dec = []   
        data_size = 512
        
        for i in range(math.ceil(encode_input.shape[0]/data_size)):
            errortypes, decoded = pfr.decode(
                model,
                encode_input.tolist()[i*data_size:(i+1)*data_size], #[code1, code2]==> ['class', 'A',...], ['class' 'B' ...]
                start_token=target_token_dict['<START>'],
                end_token=target_token_dict['<END>'],
                pad_token=target_token_dict['<PAD>'],
                max_len=source_max_len
            )
            #[dec1, dec2] like[ ['class', 'A',...], ['class' 'B' ...] ]
            result_err.append(errortypes)
            result_dec.append(decoded)
         
        print("len(result_err):", len(result_err))
        print("len(result_dec):", len(result_dec))
        print("len(result_err[0]):", len(result_err[0]))
        print("len(result_dec[0]):", len(result_dec[0]))    
        
        Gate = 0.5

        #TypeOutput = drive_path + modelset + '_test_type.txt' #錯誤類型輸出
        TypeOutput = drive_path + 'checkpoint_'+ modelset + '_test_type.txt'

        with open(TypeOutput, 'w') as f :
            Sum_Recall = 0.0
            Sum_Precision = 0.0
            Sum_F_Score = 0.0
                     
            for i in range(len(result_err)):# error classify
                errortypes_gate = np.array([list(map(lambda e: 1 if np.sign(e-Gate)>0 else 0, t)) for t in result_err[i]])
                
                if encode_input.shape[0] % data_size == 0: 
                    result_err_len = data_size
                else:
                    result_err_len = data_size if i < encode_input.shape[0] // data_size else encode_input.shape[0] % data_size
                    
                for j in range(result_err_len):                                
                    Num = "===== " + str(i*data_size+j) + " ====="
                    print(Num)
                    f.write(Num+"\n")
   
                    std = "=====標準答案=====" + str(decode_output1[0][i*data_size+j])
                    print(std)
                    f.write(std+"\n")

                    pred = "=====預測答案=====" + str(errortypes_gate[j])
                    print(pred)
                    f.write(pred+"\n")

                    precision = metrics.precision_score(decode_output1[0][i*data_size+j], errortypes_gate[j])
                    Sum_Precision += precision
                    recall = metrics.recall_score(decode_output1[0][i*data_size+j], errortypes_gate[j])
                    Sum_Recall += recall
                    f_score = metrics.f1_score(decode_output1[0][i*data_size+j], errortypes_gate[j])
                    Sum_F_Score += f_score

                    precision_print = "Precision : " + str(precision)
                    print(precision_print)
                    f.write(precision_print+"\n")

                    recall_print = "Recall : " + str(recall)
                    print(recall_print)
                    f.write(recall_print+"\n")

                    f_score_print = "F score : " + str(f_score)
                    print(f_score_print)
                    f.write(f_score_print+"\n")

                    print(" ")
                    f.write("\n")

            Avg_Precision = round(Sum_Precision/encode_input.shape[0], 4) #平均預測
            Avg_Recall = round(Sum_Recall/encode_input.shape[0], 4) #平均召回
            Avg_F_Score = round(Sum_F_Score/encode_input.shape[0], 4) #平均 F 值

            Avg_Precision_print = "Avg_Precision : " + str(Avg_Precision)
            print(Avg_Precision_print)
            f.write(Avg_Precision_print+"\n")

            Avg_Recall_print = "Avg_Recall : " + str(Avg_Recall)
            print(Avg_Recall_print)
            f.write(Avg_Recall_print+"\n")

            Avg_F_Score_print = "Avg_F_Score : " + str(Avg_F_Score)
            print(Avg_F_Score_print)
            f.write(Avg_F_Score_print+"\n")

            print(" ")
            f.write("\n")

        #MessOutput = drive_path + modelset + '_test_meteor.txt' #錯誤訊息輸出
        MessOutput = drive_path + 'checkpoint_'+ modelset + '_test_meteor.txt'

        with open(MessOutput, 'w') as f :
            Sum_Meteor = 0.0
            for i in range(len(result_dec)): #for error messages OK!
                if encode_input.shape[0] % data_size == 0: 
                    result_dec_len = data_size
                else:
                    result_dec_len = data_size if i < encode_input.shape[0] // data_size else encode_input.shape[0] % data_size
                for j in range(result_dec_len):
                    Num = "===== " + str(i*data_size+j) + " ====="
                    print(Num)
                    f.write(Num+"\n")

                    standard = ''.join(target_test_tokens[i*data_size+j])
                    standard_print = "=====標準答案=====" + standard
                    ps_standard = parseSentence(standard)
                    print(standard_print)
                    f.write(standard_print+"\n")

                    predicted = ''.join(map(lambda x: target_token_dict_inv[x], result_dec[i][j][1:-1]))
                    predicted_print = "=====預測答案=====" + predicted
                    ps_predicted = parseSentence(predicted)
                    print(predicted_print)
                    f.write(predicted_print+"\n")

                    Meteor_Score = round(meteor.meteor_score([str(ps_standard)], str(ps_predicted)), 4)
                    Sum_Meteor += Meteor_Score
                    Meteor_Score_print = "Meteor_Score : " + str(Meteor_Score)
                    print(Meteor_Score_print)
                    f.write(Meteor_Score_print+"\n")

                    print(" ")
                    f.write("\n")
                
            Avg_Meteor = round(Sum_Meteor/encode_input.shape[0],4)
            Avg_Meteor_print = "Avg_Meteor_Score : " + str(Avg_Meteor)
            print(Avg_Meteor_print)
            f.write(Avg_Meteor_print+"\n")
        
        
x=TestTranslate()
x.test_translate()